{{title_its_nb5}}

{{intro}}

The previous notebooks in this tutorial demonstrated how to use Xarray to access, inspect, manipulate and analyze raster time series data at the scale of an individual glacier. In this notebook, we shift our focus to a sub-regional scale, looking at all of the glaciers within a given ITS_LIVE data cube. This workflow will draw on elements from the past notebooks while introducing new tools for examining raster data along temporal and spatial dimensions (ie. across multiple glaciers). 



::::{tab-set} 
:::{tab-item} Outline

(content:Section_A)=
**[A. Read and organize data](#a-read-and-organize-data)**
- {{a1_its_nb5}}
- {{a2_its_nb5}}

(content:Section_B)=
**[B. Combine raster and vector data to create a vector data cube](#b-combine-raster-and-vector-data-to-create-a-vector-data-cube)**
- {{b1_its_nb5}}
- {{b2_its_nb5}}
- {{b3_its_nb5}}

(content:Section_C)= 
**[C. Data visualization](#c-data-visualization)**
- {{c1_its_nb5}}
- {{c2_its_nb5}}
- {{c3_its_nb5}}
:::  
:::{tab-item} Learning Goals
{{concepts}}
- Querying and accessing raster data from cloud object storage
- Accessing and manipulating vector data
- Handling coordinate reference information
- Calculating and visualizing summary statistics

{{techniques}}
- Access cloud-hosted [Zarr](https://zarr.readthedocs.io/en/stable/) data cubes using [Xarray](https://xarray.dev/)
- Reading [GeoParquet](https://geoparquet.org/) vector data using [GeoPandas](https://geopandas.org/en/stable/)
- Rasterize vector objects using [Geocube]()
- Spatial joins of vector datasets using [GeoPandas](https://geopandas.org/en/stable/)
- Using [dask](https://www.dask.org/) to work with out-of-memory data
- Calculating summary statistics of [Xarray](https://xarray.dev/) and [Pandas](https://pandas.pydata.org/) objects
- Data visualization using [Pandas](https://pandas.pydata.org/)
- Interactive data visualization with [GeoPandas](https://geopandas.org/en/stable/)
:::
::::


{{break}}

Expand the next cell to see specific packages used in this notebook and relevant system and version information. 

In [ ]:
%xmode minimal

import contextily as cx
import dask
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import xarray as xr
import xvec

import itslivetools

%config InlineBackend.figure_format='retina'

In [ ]:
cwd = pathlib.Path.cwd()
tutorial1_dir = pathlib.Path(cwd).parent

{{break}}

## A. Read and organize data

### {{a1_its_nb5}}

As in past notebooks, we use `itslivetools.find_granule_by_point()` to query the ITS_LIVE catalog for the correct url.

In [ ]:
itslive_catalog = gpd.read_file(
    "https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json"
)
url = itslivetools.find_granule_by_point([95.180191, 30.645973])
url

Following the example shown in the initial data inspection [notebook](), we read the data in *without* dask at first so that we can lazily organize the dataset in chronological order

In [ ]:
dc = itslivetools.read_in_s3(url, chunks=None)
dc = dc.sortby("mid_date")
dc

Now we add a chunking scheme, converting the underlying Numpy arrays to Dask arrays.

In [ ]:
# First, check the preffered chunk sizes
dc["v"].encoding

In [5]:
# Then, chunk dataset
dc = dc.chunk({"mid_date": 20000, "x": 10, "y": 10})

We'll resample the time series to 3-month resolution.

In [6]:
dc_resamp = dc.resample(mid_date="3ME").mean()

Create a `crs` object based on the `projection` data variable of the data cube (`dc`) object. We'll use this later.

In [ ]:
crs = f"EPSG:{dc.projection}"
crs

### {{a2_its_nb5}}

In [ ]:
se_asia = gpd.read_parquet("../data/rgi7_region15_south_asia_east.parquet")

se_asia.head(3)

What coordinate reference system is this dataframe in? 

In [ ]:
se_asia.crs

The vector dataset is in WGS 84, meaning that its coordinates are in degrees latitude and longitude rather than meters N and E. We will project this dataset to match the projection of the raster dataset.

In [13]:
se_asia_prj = se_asia.to_crs(crs)

In [ ]:
print(len(se_asia_prj))

The vector dataframe representing glacier outlines is very large. For now, we're only interested in glaciers that lie within the footprint of the ITS_LIVE granule we're working with and that are larger than 5 square kilometers in area. We subset the full dataset to match those conditions:

To start with, we will look only at glaciers larger in area than 5km2. Subset the dataset to select for those glaciers; start by making a GeoDataFrame of the ITS_LIVE granule footprint in order to perform a spatial join and select only the glaciers from the RGI dataframe (`se_asia_prj`) that are within the granule:

In [15]:
dc_bbox = itslivetools.get_bounds_polygon(dc)
dc_bbox["Label"] = ["Footprint of ITS_LIVE granule"]

In [16]:
# Spatial join
rgi_subset = gpd.sjoin(se_asia_prj, dc_bbox, how="inner")
# Select only glaciers where area >= 5 km2
rgi_subset = rgi_subset.loc[rgi_subset["area_km2"] >= 5.0]

In [ ]:
print(f"Now, we are looking at {len(rgi_subset)} glaciers.")

## B. Combine raster and vector data to create a vector data cube

Vector data cubes are a data structure similar to a raster data cube, but with a dimension represented by an array of geometries. For a detailed explanation of vector data cubes, see Edzer Pebesma's [write-up](https://r-spatial.org/r/2022/09/12/vdc.html). [Xvec](https://xvec.readthedocs.io/en/stable/) is a relatively new Python package that implements vector data cubes within the Xarray ecosystem. This is an exciting development that can drastically simplify workflows that examine data along both spatial and temporal dimensions and involve spatial features represented by vector points, lines and polygons.  

To explain this in more detail, we currently have a raster data cube (the ITS_LIVE time series) that covers the entire spatial footprint shown in blue below. However, the locations in which we are interested in this data are the glaciers outlined in red. Working with this data as a 3-dimensional vector cube is not a very efficient way of accessing the data at the locations shown in red. 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
dc_bbox.to_crs("EPSG:3857").plot(ax=ax, alpha=0.5)
rgi_subset.to_crs("EPSG:3857").plot(ax=ax, facecolor="none", edgecolor="r")

cx.add_basemap(
    ax=ax,
    crs="EPSG:3857",
    source=cx.providers.Esri.WorldImagery,
)
fig.suptitle(
    "Spatial extent of ITS_LIVE granule, shown in blue \n outlines of glaciers of interest, shown in red."
);

Instead, we use the [`Xvec.zonal_stats()`](https://xvec.readthedocs.io/en/stable/zonal_stats.html) method to convert the 3-dimensional cube to a 2-dimensional cube that has  time dimesnion and a geometry dimension. Each element of the geometry dimension is a glacier from the `rgi_subset` dataframe.

```{note} 
Because we are working with polygon geometries, we use zonal_stats() which performs a reduction over the area of the polygon. If our vector data was made up of point features, we could use [`Xvec.extract_points()`](https://xvec.readthedocs.io/en/stable/extract_pts.html)
```

### {{b1_its_nb5}}

In [19]:
dask.config.set({"array.slicing.split_large_chunks": True})
vector_data_cube = dc_resamp.xvec.zonal_stats(
    rgi_subset.geometry,
    x_coords="x",
    y_coords="y",
).drop_vars("index")

In [ ]:
vector_data_cube

Great, now we've gone from a 3-d object with (mid_date,x,y) dimensions to a 2-d object with (mid_date, geometry) dimensions. However, in addition to the geometry data stored in the vector dataframe, we'd also like to add some of the attribute data to the ITS_LIVE time series vector cube. The following cell adds attributes as coordinate variables to the vector data cube.

### {{b2_its_nb5}}

In [21]:
# Define attributes to be added
rgi_attrs_dict = {"RGIId": "rgi_id", "Area_km2": "area_km2", "Slope_deg": "slope_deg"}


def update_cube_attrs(ds, gdf, attrs_dict):
    for k, v in attrs_dict.items():
        ds[k] = (("geometry"), gdf[v].values)
        ds = ds.assign_coords({k: ds[k]})
    return ds


vector_data_cube = update_cube_attrs(vector_data_cube, rgi_subset, rgi_attrs_dict)

In [ ]:
vector_data_cube

### {{b3_its_nb5}}

Later in this notebook, we will want to perform operations that require this object to be loaded into memory. All of our steps thus far have happened lazily, and when we do load the data into memory it will be quite time-consuming. 

To skip this step, I've loaded the data into memory and written it to disk. This allows us to read the vector data cube (which is much more manageable) into memory from file. 

Because this step is quite computationally intensive, I won't execute it here, but I will include the code for the sake of completeness. 

#### Steps to write vector data cube to disk
1. Encode vector data code geometries as [CF] geometries. 
This is required because Shapely geometries (how geometries are represented in memory) are not compatible with array-based file formats such as Zarr. To get around this, Xvec uses a package called cf-xarray to encode the Shapely geometries as CF geometries. For more detail on reading and writing vector data cubes, see the Xvec [documentation](https://xvec.readthedocs.io/en/stable/io.html).

    ```python
    encoded_vdc = vector_data_cube.xvec.encode_cf()
    ```

2. Write encoded data cube to Zarr (*This triggers `.compute()`, loading the data into memory before writing it to disk.* On my computer, this took about 12 minutes. 
    ```python
    with ProgressBar():

        encoded_vdc.to_zarr('../data/regional_glacier_velocity_vector_cube.zarr', mode='w')
    ```
    Output:
    [########################################] | 100% Completed | 12m 15s

## C. Data visualization

### {{c1_its_nb5}}

The vector data cube is stored on disk with CF geometries. We go through the reverse of the process we used to write the object to disk, decoding the CF geometries to Shapely geometries with `xvec.decode_cf()`. 

In [23]:
vector_data_cube_cf = xr.open_zarr(
    os.path.join(tutorial1_dir, "data/regional_glacier_velocity_vector_cube.zarr")
)
vector_data_cube = vector_data_cube_cf.xvec.decode_cf().compute()

### {{c2_its_nb5}}

Xvec has a method, `to_geodataframe()` that allows us to easily convert the `xr.Dataset` vector cube to a `gpd.GeoDataFrame`. We can then use the GeoPandas `.explore()` method to interactively visualize the data.

We will look at mean velocities over time.

In [ ]:
vector_data_cube_mean = vector_data_cube.mean(dim="mid_date")

vector_data_cube_mean["vmag"] = np.sqrt(
    vector_data_cube_mean["vx"] ** 2 + vector_data_cube_mean["vy"] ** 2
)

vector_data_cube_mean["vmag"].xvec.to_geodataframe(geometry="geometry").explore("vmag")

We could also look at single seasons:

In [ ]:
vector_cube_seasonal_mean = vector_data_cube.groupby("mid_date.season").mean()

vector_cube_seasonal_mean["vmag"] = np.sqrt(
    vector_cube_seasonal_mean["vx"] ** 2 + vector_cube_seasonal_mean["vy"] ** 2
)

vector_cube_seasonal_mean.sel(season="JJA")["vmag"].xvec.to_geodataframe(
    geometry="geometry"
).explore("vmag")

### {{c3_its_nb5}}

In addition to using GeoPandas plotting features, we can still use Xarray's built in plotting features to visualize the non-spatial elements of the data. Below, we visualize the relationships between magnitude of velocity (y-axis), glacier slope (x-axis) and glacier area (hue). 

In [ ]:
fig, ax = plt.subplots()

fig.suptitle("Mean velocity in the context of glacier area and slope", fontsize=16)

sc = vector_data_cube_mean.plot.scatter(
    x="Slope_deg",
    y="vmag",
    hue="Area_km2",
    ax=ax,
    add_colorbar=True,
    cbar_kwargs={"label": "Glacier area (sq. km.)"},
)
ax.set_ylabel("Magnitude of velocity (m/yr)", fontsize=14)
ax.set_xlabel("Glacier slope (degrees)", fontsize=14);

{{conclusion}}